In [ ]:
# just in case you are running python 2
from __future__ import print_function

# just for viewing images
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Compare performance and ease of use between Tensorflow and keras

### By the end of this demo you will have:
- built 3 neural networks
- solved a multiclass classification problem
- approached state of the art performance on MNIST

# Tensorflow

If you have a tensorflow supported GPU
```
pip install tensorflow-gpu
```
If you dont (most Mac Users)
```
pip install tensorflow
```

<a href="https://www.tensorflow.org/versions/r0.11/get_started/os_setup">TensorFlow Install Help</a>

<a href="https://www.tensorflow.org/">Tensorflow Docs</a> <--> Really good documentation on NN's

### This demo was built off of Tensorflow version 0.12.1

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

def preview_tf_mnist(n):
    """Just to view the training data"""
    plt.imshow(mnist.train.images[n].reshape(28,28),
           cmap='gray')
    print('True Label:',np.argmax(mnist.train.labels[n]))
    plt.show()
    
def evaluate_tf_mnist(n,prediction):
    """Just to view the training data"""
    plt.imshow(mnist.test.images[n].reshape(28,28),
           cmap='gray')
    print('True Label:',np.argmax(mnist.test.labels[n]))
    print('Prediction:',prediction)
    plt.show()

In [ ]:
preview_tf_mnist(135)

### Simple no hidden layer NN

<img src="simple_nn.png" width="400px">

In [ ]:
tf.__version__

In [ ]:
# create new tensorflow session
sess = tf.InteractiveSession()

# create placeholders for inputs and outputs
x = tf.placeholder(tf.float32, shape=[None,784])
y_ = tf.placeholder(tf.float32, shape=[None,10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# initialize variables into session
# this variable initializer is different for older versions
sess.run(tf.global_variables_initializer())

y = tf.matmul(x,W) + b

In [ ]:
# loss function
cross_entropy = tf.reduce_mean(\
                tf.nn.softmax_cross_entropy_with_logits(y,y_)\
                              )

In [ ]:
# optimizer
train_step = tf.train.GradientDescentOptimizer(0.5)\
                .minimize(cross_entropy)

In [ ]:
# train the net
# 1000 training iterations
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

In [ ]:
# evaluate performance
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:',
      accuracy.eval(feed_dict={x: mnist.test.images,
                               y_: mnist.test.labels}))

Not too bad 92% accuracy for a 10 class classification problem in less than 15 lines of code

# keras
```
pip install keras
```
requires either tensorflow or theano installation

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [ ]:
model = Sequential()
model.add(Dense(10,input_dim=784,activation='softmax'))
sgd = SGD(0.5)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(mnist.train.images,mnist.train.labels, 
          batch_size=100, nb_epoch=5, 
          validation_data=(mnist.test.images,mnist.test.labels))

In [ ]:
score = model.evaluate(mnist.test.images,
                       mnist.test.labels,
                       verbose=0)
print('Accuracy:',score[1])

Same results as TensorFlow, but half the amount of code needed.

### What performance can we get using keras in 10 lines of code?

Intro convolutional Neural Nets

In [ ]:
train_2d = mnist.train.images.reshape(len(mnist.train.images),28,28,1)
test_2d = mnist.test.images.reshape(len(mnist.test.images),28,28,1)

In [ ]:
from keras.layers import Convolution2D, MaxPooling2D, Flatten

In [ ]:
model = Sequential()
model.add(Convolution2D(32,5,5,
                        input_shape=(28,28,1),
                        activation='relu',
                       init='normal'))
model.add(MaxPooling2D((2,2)))
model.add(Convolution2D(64,5,5,activation='relu',init='normal'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024,activation='relu',init='normal'))
model.add(Dense(10))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(train_2d,mnist.train.labels, 
          batch_size=100, nb_epoch=5, 
          validation_data=(test_2d,mnist.test.labels))

In [ ]:
score = model.evaluate(test_2d,
                       mnist.test.labels,
                       verbose=0)
print('Accuracy:',score[1])

98% accuracy in 10 lines of code on a 10 class classification problem.

Identical Tensorflow NN is about 35 lines of code.

### Visualize Convolution Kernels

In [ ]:
def plot_filters(layer):
    w = layer.get_weights()[0][:,:,0,:]
    print(w.shape)
    for k in range(w.shape[-1]):
        kernel = w[:,:,k]
        plt.imshow(kernel,cmap='gray',
                   origin='lower',
                   interpolation=None)
        plt.show()

In [ ]:
conv1 = model.layers[0]
plot_filters(conv1)

In [ ]:
conv2 = model.layers[2]
plot_filters(conv2)

# Takeaway

- Neural nets are an incredibly powerful ML tool
- keras provides a nice API built on top of tensorflow or theano
- building NN in keras is just as easy as scikit learn, in fact keras provides an interface with scikit learn, so you can use NNs just like any other ML algorithm.
